# 2023-09-05 · Inhibitory impulse response (PSP)

We've had excitatory (https://tfiers.github.io/phd/nb/2023-07-26__AdEx_Nto1_we_I_syn.html#impulse-response), \
but inh is interesting too: same syn strength, same PSP? Or no.

In [1]:
include("lib/Nto1.jl")

using Revise … ✔ (0.3 s)
using Units, Nto1AdEx, ConnectionTests, ConnTestEval … ✔
using StatsBase … ✔ (0.2 s)
